# Commercial data ordering through Sentinel Hub

This notebook can be used to order commercial high resolution satellite data from the following data providers or missions:

* AIRBUS [Pleiades](https://docs.sentinel-hub.com/api/latest/data/airbus/pleiades/) & [SPOT](https://docs.sentinel-hub.com/api/latest/data/airbus/spot/)
* [Planet SCOPE](https://docs.sentinel-hub.com/api/latest/data/planet/planet-scope/)
* [Planet SkySat](https://docs.sentinel-hub.com/api/latest/data/planet/skysat/)
* [WorldView](https://docs.sentinel-hub.com/api/latest/data/maxar/world-view/)

A Sentinel Hub(https://www.sentinel-hub.com/) account with suffcient credit is required. You'll need to provide your client id and client secret or set the relevant environment variables.

## Load Python packages and configure Sentinel Hub connection

> Additional python packages may need to be installed in your enviroment, e.g. `sentinelhub` or `area`.

In [1]:
#!pip install sentinelhub
#!pip install area

> For use outside of the Digital Earth Africa sandbox

In [2]:
#!pip install deafrica_tools

In [4]:
from sentinelhub import SHConfig
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from odc_sh import SentinelHubCommercialData
from odc_sh import Providers, AirbusConstellation, ThumbnailType, WorldViewKernel, WorldViewSensor, SkySatType, SkySatBundle, ScopeType, ScopeBundle
import pandas as pd
import numpy as np
import folium
from shapely.geometry import box
from area import area
import json
import shapely
import io
from PIL import Image
import matplotlib.pyplot as plt
import os

from deafrica_tools.plotting import display_map, map_shapefile, _degree_to_zoom_level

sh_client_id=""
sh_client_secret=""

if not sh_client_id:
    sh_client_id = os.environ['SH_CLIENT_ID']

if not sh_client_secret:
    sh_client_secret = os.environ['SH_CLIENT_SECRET']



config = SHConfig()
config.sh_client_id = sh_client_id
config.sh_client_secret = sh_client_secret


shcd = SentinelHubCommercialData(config)

KeyError: 'SH_CLIENT_ID'

### SentinelHubCommercialData package

The SentinelHubCommercialData package is used to send and retrieve information from the Sentinel Hub API.
It provides functionalities to

* Check account quotas
* Search for available commercial data
* Configure, delete, and confirm an order
* Check order status

Use of these functions are explained in details below.

### Printing information from API responses

Every response that has method "print_info()", also has a raw data in response object.
Example:

In [ ]:
q = shcd.quotas()
q.print_info()

In [ ]:
# response data
print(q.data)

## Getting quotas

Check the quota and usage status for the Sentinel Hub account. Determine if there's suffcient credit to continue the ordering of desired imagery.

**Fang's comment: not sure how to check quota for a single collection. What id should be used? Using "AIRBUS_PLEIADES" returns an error.**

In [ ]:
# Optional 1: To get single quota add id as parameter
#
#     shcd.quotas("asd124-12ddas...")
#

q = shcd.quotas()
q.print_info()


## Define simple search parameters

A time period and an area of interest are used in the search for imagery.

### Setting date and time 

For the start and end of the period of interest.

In [ ]:
time_from = "2020-01-01T00:00:00Z"
time_to = "2022-12-31T23:59:59Z"

### Setting area of interest 

The search area can be defined in one of the 2 options:

1. Bounding Box
2. Polygon

### 1. Bounding Box

If a bounding box is used, the longitude and latitude bounds are provided as a list, in the order of `[min_lon, min_lat, max_lon, max_lat]`.

In [ ]:
# Option 1: Set bounds as bbox

bounds = [
    30.292,
    -1.545,
    30.386,
    -1.460
]


**Check the area size**

In [ ]:
aoi = box(bounds[0],bounds[1],bounds[2],bounds[3])

#bbox area calculation
area_sqm = area(json.dumps(shapely.geometry.mapping(aoi))) # area in m2 (sqm)

print('Area for selected bounding box is:', round(area_sqm * 10 ** (-6),2), 'sq. km')

**Display bounding box on the map**

In [ ]:
x = (bounds[0],bounds[2])
y = (bounds[1],bounds[3])

#view the location
display_map(x=x, y=y)

### 2. Polygon

Alternatively, a polygon can be provided as a geojson string.

In [ ]:
# Option 2: Set bounds as Polygon
bounds_polygon = {
  "type": "Polygon",
  "coordinates": [
   [
    [
     12.500395,
     41.931337
    ],
    [
     12.507856,
     41.931018
    ],
    [
     12.507513,
     41.927825
    ],
    [
     12.50048,
     41.928719
    ],
    [
     12.500395,
     41.931337
    ]
   ]
  ]
 }

**Display polygon on the map**

World imagery from ESRI is used as the basemap by default.

In [ ]:
# Extract coordinates from JSON

geom = bounds_polygon["coordinates"][0]

# Calculate polygon centroid to center the map

lat_all = []
for i in range(len(geom)):
    lat_all.append(geom[i][1])
    
lon_all = []
for i in range(len(geom)):
    lon_all.append(geom[i][0])
    
center = [np.mean(lat_all), np.mean(lon_all)]

# Calculate zoom level based on coordinates

lat_zoom_level = _degree_to_zoom_level(min(lat_all), max(lat_all))
lon_zoom_level = _degree_to_zoom_level(min(lon_all), max(lon_all))

zoom_level = min(lat_zoom_level, lon_zoom_level)

In [ ]:
#Multiple options for basemap
#Option 1. World imagery from ESRI
World_Imagery = ("http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer" + '/MapServer/tile/{z}/{y}/{x}')

#Option 2: World Topo Map from ESRI
#World_Topo_Map = (
#    'http://services.arcgisonline.com/arcgis/rest/services/World_Topo_Map'
#    + '/MapServer/tile/{z}/{y}/{x}'
#)

#The attribute paramater needs to be provided for custom tiles
map = folium.Map(location=center, tiles=World_Imagery, attr='ESRI World Imagery', zoom_start=zoom_level)
folium.GeoJson(bounds_polygon, name="geojson").add_to(map)

map

## Search for available imagery

Choose one of the providers below to run the search. 

The search will return available imagery and properties that might help decide which images to order. The list of properties available are different for different data collections. 

Each image has a unique id which will be used to identify them in the ordering process.

### 1. AIRBUS Pleiades & SPOT

For more information on mission and data characteristics: AIRBUS [Pleiades](https://docs.sentinel-hub.com/api/latest/data/airbus/pleiades/) & [SPOT](https://docs.sentinel-hub.com/api/latest/data/airbus/spot/)

In addition to the time period and area of interest, additional optional query parameters can be used: 

* maxCloudCoverage (Values: 0-100 | 100 as default)
* maxSnowCoverage (Vaules: 0-90 | 90 as default)
* maxIncidenceAngle (Values: 0-90 | 90 as default)
* processingLevel (Values: "Sensor","Album" | "Sensor" as default)

In [ ]:
# Option 1: AIRBUS Pleiades & SPOT
# Optional parameters: 
#    - maxCloudCoverage (Values: 0-100 | 100 as default)
#    - maxSnowCoverage (Vaules: 0-90 | 90 as default)
#    - maxIncidenceAngle (Values: 0-90 | 90 as default)
#    - processingLevel (Values: "Sensor","Album" | "Sensor","Album" as default)
#
#  example: shcd.search_airbus(AirbusConstellation, Bounds, Time_From, Time_To, *Optional parameters*)

res = shcd.search_airbus(AirbusConstellation.SPOT, bounds, time_from, time_to, maxCloudCoverage=20, maxSnowCoverage=50)

Print information about available imagery, including all or a selected list of properties.

In [ ]:
# Optional parameters: 
#    - props
#
# example: res.print_info(props=["id", "acquisitionDate", "resolution", "cloudCover"])
res.print_info()

In [ ]:
res.query

#### Select suitable images

Select images with desired properties. Record the unique ids of the selected images.

In [ ]:
# Getting ids
item_ids = res.get_ids()

selected_ids = [item_ids[4], item_ids[6]] if len(res.data.features) else []
selected_ids

### 2. Planet SCOPE

For more information on mission and data characteristics: [Planet SCOPE](https://docs.sentinel-hub.com/api/latest/data/planet/planet-scope/)

In addition to the time period and area of interest, additional optional query parameters can be used: 

* maxCloudCoverage (Values: 0-100 | 100 as default)

In [ ]:
# Option 2: Planet SCOPE
# Optional parameters: 
#    - maxCloudCoverage (Values: 0-100 | 100 as default)
#
#  example: shcd.search_airbus(ScopeType, ScopeBundle, Bounds, Time_From, Time_To, *Optional parameters*)

res = shcd.search_planet(ScopeType.PSScene, ScopeBundle.ANALYTIC_UDM2, bounds, time_from, time_to, maxCloudCoverage=90)

Print information about available imagery, including all or a selected list of properties.

In [ ]:
# Optional parameters: 
#    - props
#
# example: res.print_info(props=["cloud_cover", "snow_ice_percent", "acquired", "pixel_resolution"])
res.print_info()

#### Select suitable images

Select images with desired properties. Record the unique ids of the selected images.

In [ ]:
# Getting ids
item_ids = res.get_ids()

selected_ids = [item_ids[0], item_ids[1]] if len(res.data.features) else []
selected_ids

### 3. Planet SkySat

For more information on mission and data characteristics: [Planet SkySat](https://docs.sentinel-hub.com/api/latest/data/planet/skysat/)


In [ ]:
# Option 3: Planet SkySat
#
#  example: shcd.search_airbus(ScopeType, ScopeBundle, Bounds, Time_From, Time_To, planetApiKey=<your_planey_api_key>)

res = shcd.search_planet(SkySatType.SkySatCollect, SkySatBundle.PANCHROMATIC, bounds, time_from, time_to, planetApiKey="")

Print information about available imagery, including all or a selected list of properties.

In [ ]:
# Optional parameters: 
#    - props
#
# example: res.print_info(props=["cloud_cover", "snow_ice_percent", "acquired", "pixel_resolution"])
res.print_info()

#### Select suitable images

Select images with desired properties. Record the unique ids of the selected images.

In [ ]:
# Getting ids
item_ids = res.get_ids()

selected_ids = [item_ids[0], item_ids[1]] if len(res.data.features) else []
selected_ids

### 4. WorldView

For more information on mission and data characteristics: [WorldView](https://docs.sentinel-hub.com/api/latest/data/maxar/world-view/)

In addition to the time period and area of interest, additional optional query parameters can be used: 

* maxCloudCoverage (Values: 0-100 | 100 as default)
* minOffNadir (Values: 0-45 | 0 as default)
* maxOffNadir (Values: 0-45 | 45 as default)
* minSunElevation (Values: 0-90 | o as default)
* maxSunElevation (Values: 0-90 | 90 as default)
* sensor (Values: WorldViewSensor | Any as default)


In [ ]:
# Option 4: WorldView MAXAR
# Optional parameters:
#    - maxCloudCoverage (Values: 0-100 | 100 as default)
#    - minOffNadir (Values: 0-45 | 0 as default)
#    - maxOffNadir (Values: 0-45 | 45 as default)
#    - minSunElevation (Values: 0-90 | o as default)
#    - maxSunElevation (Values: 0-90 | 90 as default)
#    - sensor (Values: WorldViewSensor | Any as default)
#
#  example: shcd.search_airbus(WorldViewKernel, ScopeBundle, Bounds, Time_From, Time_To, *Optional parameters*)
#

res = shcd.search_worldview(WorldViewKernel.MTF, bounds, time_from, time_to, sensor=WorldViewSensor.WV01.value)

Print information about available imagery, including all or a selected list of properties.

In [ ]:
# Optional parameters: 
#    - props
#
# example: res.print_info(props=["catalogId", "sensor", "maxSunAzimuth", "acquisitionDateStart"])
res.print_info()

#### Select suitable images

Select images with desired properties. Record the unique ids of the selected images.

In [ ]:
# Getting ids
item_ids = res.get_ids()

selected_ids = [item_ids[0], item_ids[1]] if len(res.data.features) else []
selected_ids

## Display thumbnail for selected images

Thumbnails are provided for the entire scene, which may cover regions outside of the area of interest. At a coarse resolution, they can be used to visually check overall image quality before ordering. 

In [ ]:
# Define product ids for a preview
if not len(selected_ids):
    print("No ids found.")

for sid in selected_ids:
    thumbnail = shcd.thumbnail(res.thumb, sid)
    image_bytes = io.BytesIO(thumbnail.content)
    image = Image.open(image_bytes)
    plt.imshow(image)
    plt.show()
    

## Data Order

Proceed to order the data if satisfactory images are found in the search above.

### OPTIONAL: Getting compatible data collections

Data can be organized in collections for ease of management. New data can be added to exisiting collections if the unique id for the collection is provided during order.

In [ ]:
collections = shcd.get_collection(res.query)
collections.print_info()

#### Record unique id for a collection to be used in the ordering process

In [ ]:
colIdx = 0 # Idx number from search above
collectionId = collections.data[colIdx]["id"]
collectionId

### Create an query order: Airbus, Planet SkySat and Scope

Order all data that satisfy query criteria.

In [ ]:
# Optional parametrs:
#  - collectionId
#
# example: shcd.order("Order name", Query, *Optional_parameters*)

#response = shcd.order("New query order", res.query)

### Create a normal order: Airbus, Planet SkySat and Scope

Order data that satisfy query criteria and match specified unique ids.

To add data to an exisiting collection, use the optional parameter `collectionId`.

In [ ]:
# Optional parametrs:
#  - collectionId
#
# example: shcd.order("Order name", Query, Items_Ids)
# example with collectionId: shcd.order("Order name", Query, Items_Ids, collectionId=collectionId)


order = shcd.order("New normal order", res.query, item_ids=selected_ids)
order.print_info()

> An order would have been created but not yet executed. A confirmation step is required for the order execution.

### Confirming or delete an order

#### Getting status of orders

Retrieve the list of current and previous orders and their status.

In [ ]:
# Optional: Get single order by adding order_id as parameter
# example: shcd.get_orders("asdf12-12bfa...")

orders = shcd.get_orders()
orders.print_info()


#### Select unique id of an order

In [ ]:
order_ids = orders.get_ids()
order_id = order_ids[0] #you can use order_ids[1]

### Delete an order

An order can be deleted using its unique id.

In [ ]:
res = shcd.delete_order(order_id)
res.print_info()

### Confirm an order

**Proceed to order imagery!**

Once confirmed, an order will be executed. The status of the order should be changed to `RUNNING` shortly after confirmation. The time it takes for an order to complete will depend on amount and type of data ordered. Data is available for use when the order status is `DONE`.

In [ ]:
res = shcd.confirm_order(order_id)
res.print_info()